In [1]:
import os
import torch
from torch.onnx import export
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration
from kobart import get_kobart_tokenizer

In [2]:
tokenizer = get_kobart_tokenizer()

sentence_text = '요약을 위해 작성하는 글입니다. 사실 이 글의 요점은 아무 주제도 없다는 것에 있습니다. 부디 기계가 이 요점을 잘 정리해서 보여주었으면 좋겠습니다.'
max_length = 1024

model_input = tokenizer(
            sentence_text,
            return_tensors='np',
            truncation=True,
            max_length=max_length,
            padding="max_length",
            add_special_tokens=True
        )
input_ids = model_input["input_ids"]
attention_mask = model_input["attention_mask"]

tokens_tensor = torch.tensor(input_ids)
segments_tensors = torch.tensor(attention_mask)

class Pytorch_to_TorchScript(torch.nn.Module):
  def __init__(self):
    super(Pytorch_to_TorchScript, self).__init__()
    self.model = BartForConditionalGeneration.from_pretrained('./kobart_summary').cuda()
  def forward(self, data, attention_mask=None):
    return self.model.generate(data.cuda(), num_beams=4,  max_length=max_length,  eos_token_id=1)


using cached model. /workspace/git/KoBART-summarization/.cache/kobart_base_tokenizer_cased_cf74400bce.zip


In [3]:
# Creating the trace
pt_model = Pytorch_to_TorchScript().eval()
traced_model = torch.jit.trace(pt_model, (tokens_tensor, segments_tensors))
torch.jit.save(traced_model, "model.pt")

/usr/local/lib/python3.8/dist-packages/transformers/models/bart/modeling_bart.py:212: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_weights.size() != (bsz * self.num_heads, tgt_len, src_len):
/usr/local/lib/python3.8/dist-packages/transformers/models/bart/modeling_bart.py:218: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attention_mask.size() != (bsz, 1, tgt_len, src_len):
/usr/local/lib/python3.8/dist-packages/transformers/models/bart/modeling_bart.py:249: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be i

/usr/local/lib/python3.8/dist-packages/transformers/generation_beam_search.py:369: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if len(self) < self.num_beams or score > self.worst_score:
/usr/local/lib/python3.8/dist-packages/transformers/generation_beam_search.py:372: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  sorted_next_scores = sorted([(s, idx) for idx, (s, _) in enumerate(self.beams)])
/usr/local/lib/python3.8/dist-packages/transformers/generation_beam_search.py:297: TracerWarning: Converting a tensor to a Python boolean might cause the trace 

In [4]:
loaded_model = torch.jit.load("model.pt")
loaded_model.eval()

RecursiveScriptModule(
  original_name=Pytorch_to_TorchScript
  (model): RecursiveScriptModule(
    original_name=BartForConditionalGeneration
    (model): RecursiveScriptModule(
      original_name=BartModel
      (shared): RecursiveScriptModule(original_name=Embedding)
      (encoder): RecursiveScriptModule(
        original_name=BartEncoder
        (embed_tokens): RecursiveScriptModule(original_name=Embedding)
        (embed_positions): RecursiveScriptModule(original_name=BartLearnedPositionalEmbedding)
        (layers): RecursiveScriptModule(
          original_name=ModuleList
          (0): RecursiveScriptModule(
            original_name=BartEncoderLayer
            (self_attn): RecursiveScriptModule(
              original_name=BartAttention
              (k_proj): RecursiveScriptModule(original_name=Linear)
              (v_proj): RecursiveScriptModule(original_name=Linear)
              (q_proj): RecursiveScriptModule(original_name=Linear)
              (out_proj): RecursiveSc

In [5]:
tokenizer = get_kobart_tokenizer()
max_length = 1024

sentence_text = """‘왜 한국에서 유리천장을 뚫은 여성 최고경영자(CEO)와 임원 중 외국계 기업 출신이 많을까?’

오랫동안 품은 의문이다. 통계적 증명이나 깊이 있는 연구 결과는 아니지만 한 가지 가설이 가능하다. 한국 기업과 외국 기업의 문화 차이, 좀 더 구체적으론 일과 가정의 양립이 제도적으로, 그리고 실질적으로 보장되느냐 여부가 아닐까 싶다. 출산과 육아가 커리어의 공백이 되는 한국. 이 문제가 한국이 초저출산 국가가 된 배경 중 하나임은 분명해 보인다.

한국은 일론 머스크 테슬라 CEO까지 ‘인구 붕괴’를 걱정할 정도로 저출산 문제가 심각하다. 취업률 하락, 집값 폭등 등 경제적인 이유와 함께 가치관 변화, 특히 여성들의 인식 변화가 출산율 급감에 영향을 미쳤을 것이다. 출산율은 각종 대책에도 불구하고 2016년 이후 뚝뚝 떨어지고 있다. 이른바 MZ세대, 그중 M(밀레니얼)세대가 결혼 및 출산 시기에 접어드는 시점과 맞물린다.

그들은 왜 출산을 기피할까. 그들의 엄마 세대는 공부도 할 만큼 했고, 사회생활을 하고 싶어 했다. 하지만 그중 많은 사람이 출산·육아 때문에 포기할 수밖에 없었다. 그런 엄마들로부터 딸들이 들었던 말은 이런 것이다. “너는 하고 싶은 거 하면서 살아라.” 아들들의 생각도 달라졌다. 아빠 세대의 나 홀로 부양 부담이 싫어졌다. 20대 절반은 결혼 후 아이 없이 사는 것에 동의한다는 설문조사 결과가 있다.

그 결과가 대한민국 합계 출산율 0.81명이다. 우리나라 여성 한 명이 평생 낳을 것으로 예상되는 평균 출생아 수가 1명도 안 된다는 얘기다. 경제협력개발기구(OECD) 38개 회원국 가운데 합계출산율이 1명에도 못 미치는 나라는 한국이 유일하다. 지금 같은 추세면 50년 뒤엔 사회적으로 젊은 사람 1명이 노인 1명을 부양해야 한다. 저출산·고령화가 심각하다는 것엔 사회적 공감대가 형성돼 있다. 문제는 여러 상황이 원인이자 결과로 뒤얽혀 있어 해법 찾기가 쉽지 않다는 것이다.

결혼과 출산은 지극히 사적인 영역이고, 개인의 선택이다. 당장 사람들의 생각과 가치관을 바꾸긴 쉽지 않다. 그렇기 때문에 저출산 대책은 마음은 있지만 여건 때문에 자녀 갖기를 망설이는 이들의 애로사항을 풀어주는 데서 시작해야 한다.

모든 정책과 제도는 만드는 것만큼이나 정착을 위한 디테일이 중요하다. 저출산 대책도 마찬가지다. 아이돌봄 서비스 같은 경우 당장 필요할 때 이용하기 어렵다는 불만이 많다. 여전히 공급이 부족하고 어딘가에 허점이 있기 때문일 것이다. 교육부 보건복지부 여성가족부 등 각 부처 정책의 연계성 등을 종합적으로 들여다볼 필요가 있다.

기업 문화가 달라져야 한다는 것은 직접적인 과제다. 저출산은 기업에도 남의 일이 아니다. 인재 풀과 시장이 쪼그라드는 문제다. 육아휴직과 연월차 휴가만 필요할 때 쓸 수 있어도 육아에 대한 부담이 훨씬 줄어들 것이다. 외국계 기업에 여성 임원이 많은 것은 우연이 아니다.

특히 남성 육아휴직에 대한 인식이 달라져야 한다. 출생아 부모 가운데 육아휴직을 쓰는 남성 비중은 4.1%에 불과하다. 여성은 65.2%다. 남성 육아휴직의 확산은 여성들이 아이를 낳으면 육아는 결국 자신의 몫이 될 것이란 부담과 경력단절, 또는 동료보다 뒤처질 것이란 두려움을 완화시켜 줄 수 있을 것이다. 모든 일이 그렇듯 기업 최고경영자의 의지와 메시지가 중요하다. 정부도 기업 문화가 변할 수 있도록 강력한 인센티브를 줘야 한다.

박성완 논설위원겸 경제교육연구소장 psw@hankyung.com"""
model_input = tokenizer(
            sentence_text,
            return_tensors='np',
            truncation=True,
            max_length=max_length,
            padding="max_length",
            add_special_tokens=True
        )
input_ids = model_input["input_ids"]
attention_mask = model_input["attention_mask"]

tokens_tensor = torch.tensor(input_ids)
segments_tensors = torch.tensor(attention_mask)

using cached model. /workspace/git/KoBART-summarization/.cache/kobart_base_tokenizer_cased_cf74400bce.zip


In [6]:
dummy_input = [tokens_tensor, segments_tensors]
result = loaded_model(*dummy_input)

In [7]:
tokenizer.decode(result.squeeze().tolist(), skip_special_tokens=True)

'한국 기업과 외국 기업의 문화 차이, 좀 더 구체적으론 일과 가정의 양립이 제도적으로, 그리고 실질적으로 보장되느냐 여부가 한국 초저출산 국가가 된 배경 중 하나이며, 한국 기업과 외국 기업의 문화 차이, 좀 더 구체적으론 일과 가정의 양립이 제도적으로, 그리고 실질적으로 보장되느냐 여부가 아닐까 싶다. \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n한국은 일론 머스크 테슬라 CEO까지 ‘인구 붕괴’를 걱정할 정도로 저출산 문제가 심각해 \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\